In [22]:
import pandas as pd

df = pd.read_csv("data/amazon_sales_2025_INR.csv")


print(df.head())

df['Date'] = pd.to_datetime(df['Date'])
df['Payment_Method'] = df['Payment_Method'].fillna('Unknown')
df['Review_Rating'] = df['Review_Rating'].fillna(df['Review_Rating'].median())

df['Product_Category'] = df['Product_Category'].str.strip().str.title()
df['Delivery_Status'] = df['Delivery_Status'].str.strip().str.title()
df['State'] = df['State'].str.strip().str.title()


df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

df['Profit_INR'] = df['Total_Sales_INR'] * 0.10

df['High_Value_Order'] = df['Total_Sales_INR'] > 100000


df_delivered = df[df['Delivery_Status'] == 'Delivered']




    Order_ID        Date Customer_ID Product_Category  Product_Name  Quantity  \
0  ORD100000  2025-01-25    CUST2796   Home & Kitchen  Cookware Set         2   
1  ORD100001  2025-08-28    CUST9669           Beauty    Hair Dryer         1   
2  ORD100002  2025-02-27    CUST5808      Electronics        Tablet         3   
3  ORD100003  2025-02-24    CUST5889      Electronics    Headphones         5   
4  ORD100004  2025-06-15    CUST9005         Clothing         Saree         5   

   Unit_Price_INR  Total_Sales_INR    Payment_Method Delivery_Status  \
0        25574.41         51148.82       Credit Card        Returned   
1        19361.41         19361.41        Debit Card        Returned   
2        38476.22        115428.66  Cash on Delivery       Delivered   
3        38145.72        190728.60       Credit Card       Delivered   
4        45940.98        229704.90               UPI       Delivered   

   Review_Rating         Review_Text      State Country  
0              1      

In [12]:
from sqlalchemy import create_engine, text

engine = create_engine("mysql+mysqlconnector://root:@localhost/amazon_sales_db")

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS amazon_sales_2025"))


In [ ]:
import pandas as pd
import sqlite3
import os

csv_path = "data/amazon_sales_2025_processed.csv"
db_path = "data/amazon_sales_2025.db"

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV file not found at {csv_path}")

df = pd.read_csv(csv_path)

df.drop_duplicates(inplace=True)
df.fillna({"Review_Rating": 0, "Review_Text": ""}, inplace=True)

conn = sqlite3.connect(db_path)


table_name = "amazon_sales_2025"
df.to_sql(table_name, conn, if_exists="replace", index=False)


cursor = conn.cursor()
cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
total_rows = cursor.fetchone()[0]
print(f"Data loaded into SQLite successfully! Total rows: {total_rows}")

conn.close()


Data loaded into SQLite successfully! Total rows: 15000
